# Imports

In [1]:
import os
import pandas as pd

In [52]:
#constants
filename = "df_fuel_ckan.csv"
path_to_data = os.path.join("..", "data")

elements = ["fire", "water", "air", "earth"]
element_mappings = {
    "BIOMASS" : "fire",
    "NUCLEAR" : "fire",
    "SOLAR" : "fire",
    "FOSSIL OIL" : "earth",
    "FOSSIL HARD COAL" : "earth",
    "FOSSIL GAS" : "earth",
    "HYDRO PUMPED STORAGE" : "water",
    "HYDRO RUN-OF-RIVER AND POUNDAGE" : "water",
    "WIND ONSHORE" : "air",
    "WIND OFFSHORE" : "air",
}

In [4]:
! ls {path_to_data}

cached_live_response.json  gridwatch.csv	   test_current.json
df_fuel_ckan.csv	   sample_geo_api.json
geo			   sample_historical.json


In [47]:
def get_df_from_csv(filename = "df_fuel_ckan.csv"):
    filepath = os.path.join(path_to_data, filename)
    return pd.read_csv(filepath)
    
def preprocess_df(df):
    #convert to datetime for groupby
    df['DATETIME'] = pd.to_datetime(df.DATETIME)

    #select only relevant columns
    sources_list = ['GAS', 'COAL', 'NUCLEAR', 'WIND', 'HYDRO','SOLAR','BIOMASS']#, 'IMPORTS', 'OTHER']
    df = df.set_index("DATETIME")[sources_list]

    rename_dict = {}
    for column in df.columns:
        for source, element in element_mappings.items():
            if column in source:
                rename_dict[column] = f"{column}_{element}"
                
    df = df.rename(columns=rename_dict)
    return df

,GAS_earth,COAL_earth,NUCLEAR_fire,WIND_air,HYDRO_water,SOLAR_fire,BIOMASS_fire
DATETIME,,,,,,,
2009-01-01 00:00:00+00:00,8336.0,15035.0,7099.0,279.0,246.0,0.0,0.0
2009-01-01 00:30:00+00:00,8456.0,15093.0,7087.0,258.0,245.0,0.0,0.0
2009-01-01 01:00:00+00:00,8419.0,15082.0,7074.0,232.0,246.0,0.0,0.0
2009-01-01 01:30:00+00:00,8276.0,15022.0,7064.0,214.0,246.0,0.0,0.0
2009-01-01 02:00:00+00:00,8247.0,14986.0,7051.0,197.0,246.0,0.0,0.0
...,...,...,...,...,...,...,...
2024-06-26 09:00:00+00:00,8110.0,0.0,5251.0,3147.0,139.0,7366.0,1653.0
2024-06-26 09:30:00+00:00,7836.0,0.0,5251.0,3280.0,74.0,8169.0,1651.0
2024-06-26 10:00:00+00:00,7292.0,0.0,5257.0,3541.0,71.0,8861.0,1653.0


In [59]:
"""
#goals
 "2009": {
    "fire": {
      "sources": [
        {
          "name": "BIOMASS",
          "value": 1292.0
        },
        {
          "name": "SOLAR",
          "value": 8429.0
        },
"""

'\n#goals\n "2009": {\n    "fire": {\n      "sources": [\n        {\n          "name": "BIOMASS",\n          "value": 1292.0\n        },\n        {\n          "name": "SOLAR",\n          "value": 8429.0\n        },\n'

In [61]:
def build_historical_output(df):
    yearly_df = df.groupby(df.index.year).sum()
    
    output = {}

    for year in yearly_df.index:
        year_data = {}
        for element in elements:
            elem_sources = [col for col in yearly_df if col.endswith(element)]
            sources = []
            for source in elem_sources:
                name = source.split('_')[0]
                value = yearly_df.loc[year][source]
                sources.append({"name" : name , "value" : value})
            year_data[element] = {"sources" : sources}
        output[year] = year_data
        
    return output
# build_output(hist_df)

In [62]:
def get_historical_output():
    hist_df = preprocess_df(get_df_from_csv())
    hist_out = build_historical_output(hist_df)

    return hist_out
get_historical_output()

{2009: {'fire': {'sources': [{'name': 'NUCLEAR',
     'value': np.float64(130004163.0)},
    {'name': 'SOLAR', 'value': np.float64(0.0)},
    {'name': 'BIOMASS', 'value': np.float64(0.0)}]},
  'water': {'sources': [{'name': 'HYDRO', 'value': np.float64(7116428.0)}]},
  'air': {'sources': [{'name': 'WIND', 'value': np.float64(8314691.0)}]},
  'earth': {'sources': [{'name': 'GAS', 'value': np.float64(294830710.0)},
    {'name': 'COAL', 'value': np.float64(197612320.0)}]}},
 2010: {'fire': {'sources': [{'name': 'NUCLEAR',
     'value': np.float64(117022087.0)},
    {'name': 'SOLAR', 'value': np.float64(0.0)},
    {'name': 'BIOMASS', 'value': np.float64(0.0)}]},
  'water': {'sources': [{'name': 'HYDRO', 'value': np.float64(4276624.0)}]},
  'air': {'sources': [{'name': 'WIND', 'value': np.float64(9209664.0)}]},
  'earth': {'sources': [{'name': 'GAS', 'value': np.float64(314185467.0)},
    {'name': 'COAL', 'value': np.float64(205679693.0)}]}},
 2011: {'fire': {'sources': [{'name': 'NUCLEAR',

## investigate percentage columns

In [34]:
perc_cols = [col for col in hist_df.columns if col.endswith("perc")]
perc_df = hist_df[perc_cols]

try_cols = ["LOW_CARBON_perc", "RENEWABLE_perc", "FOSSIL_perc"]
perc_df.loc[:,try_cols]#.sum(axis=1)
#up to solar adds up to 100

,LOW_CARBON_perc,RENEWABLE_perc,FOSSIL_perc
DATETIME,,,
2009-01-01 00:00:00+00:00,22.8,1.6,69.8
2009-01-01 00:30:00+00:00,22.6,1.5,70.0
2009-01-01 01:00:00+00:00,22.5,1.4,70.1
2009-01-01 01:30:00+00:00,22.6,1.4,70.1
2009-01-01 02:00:00+00:00,22.7,1.3,70.2
...,...,...,...
2024-06-26 09:00:00+00:00,54.0,32.8,24.9
2024-06-26 09:30:00+00:00,55.3,34.6,23.5
2024-06-26 10:00:00+00:00,57.4,36.9,21.6
